** Register 0x2c ** 

D3,D2,D1,D0 Rate Bit sdevice bandwidth and output data rate
default 0xA, 100Hz output dara rate

|Output DataRate (Hz)| Bandwidth (Hz)| Rate Code|
|:----|:----|:---|
|3200|1600|          1111 |    
|1600|      800|           1110|   
|800|       400|           1101|   
|400|       200|           1100|   
|200|       100|           1011|   
|100|       50|            1010| 
|50|        25|            1001| 
|25|        12.5|          1000| 
|12.5|      6.25|          0111| 
|6.25|      3.13|          0110| 
|3.13|      1.56|          0101| 
|1.56|      0.78|          0100| 
|0.78|      0.39|          0011| 
|0.39|      0.20|          0010| 
|0.20|      0.10|          0001| 
|0.10|      0.05|          0000| 

** Register 0x31 DATA_FORMAT (Read/Write)**

|D1| D0 |g Range|
|:-|:-|:-|
|0|  0|  ±2 g| 
|0|  1|  ±4 g| 
|1  |0|  ±8 g| 
|1  |1|  ±16 g| 

**Register 0x2D—POWER_CTL (Read/Write)**

|D3| D2| D1| D0|
|:-|:-|:-|
|Measure|  Sleep|  Wakeup|Wakeup|


In [3]:
# -*- coding: utf-8 -*-
#!/usr/bin/python

import RPi.GPIO as GPIO
import os
import numpy as np
import smbus
import time
import signal
import pickle

class ADXL345():
    def __init__(self):
        
        self.DevAdr = 0x53
        myBus = ""
        if GPIO.RPI_INFO['P1_REVISION'] == 1:
            myBus = 0
        else:
            myBus = 1
            
        self.b = smbus.SMBus(myBus)
        
        self.b.write_byte_data(self.DevAdr, 0x2C, 0x08) # BandwidthRate　originally 0B 
        self.b.write_byte_data(self.DevAdr, 0x31, 0x00) # DATA_FORMAT 10bit 2g
        self.b.write_byte_data(self.DevAdr, 0x38, 0x00) # FIFO_CTL OFF
        self.b.write_byte_data(self.DevAdr, 0x2D, 0x08) # POWER_CTL Enable
        

    
    def getOffset(self):
        data =[]
        for a in range(NO_OF_OFFSET_MEASURE):
            input = self.getData()
            data.append(input)
            time.sleep(SAMPLING_CYCLE)
        data = np.array(data)
        offset = data.mean(axis = 0)
        
        return offset

    def getData(self):
        output = []
        data = self.b.read_i2c_block_data(self.DevAdr, 0x32, 6)
        for i in range(0,6,2):
            sign = data[i + 1] & 0x80
            tmp = data[i + 1] & 0x7F
            tmp = tmp << 8
            tmp = tmp | data[i]

            if sign > 0:
                output.append(tmp - 32768)
            else:
                output.append(tmp)

        return np.array(output)

def main(arg1, arg2):
    
    global Data
    
    print "*", 
    
    ema = np.zeros(3)
    data = myADXL345.getData()
    data = np.array(data - offset)
    ema = (1 - FORGETING_FACTOR) * ema + FORGETING_FACTOR * data
    Data.append(ema)
    print "*",

    if time.time() - TIME_START >= MONITOR_TIME:
        signal.setitimer(signal.ITIMER_REAL, 0.)
        print "\nDone"
    
        with open('Data.pkl', 'wb') as f:
            pickle.dump(Data, f)
            
if __name__ == "__main__":
    
    SAMPLING_CYCLE = 0.075 # in seconds
    MONITOR_TIME = 60 # in seconds
    
    NO_OF_SAMPLES = int(MONITOR_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    print "No of samples:" , NO_OF_SAMPLES
    
    OFFSET_MEASURE_TIME = 1
    NO_OF_OFFSET_MEASURE = int(OFFSET_MEASURE_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    FORGETING_FACTOR = 1
    
    myADXL345 = ADXL345()

    offset = myADXL345.getOffset()

    Data =[]
    
    TIME_START = time.time()
    
    signal.signal(signal.SIGALRM, main)
    signal.setitimer(signal.ITIMER_REAL, 0.1, SAMPLING_CYCLE)

No of samples: 800
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

In [8]:
import pandas as pd

from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
output_notebook()

with open('Data.pkl', 'rb') as f:
    Data = pickle.load(f)

Data = list(map(list, zip(*Data)))  #Swap rows and columns

t=[float(n) * SAMPLING_CYCLE for n in range(0,NO_OF_SAMPLES)]

pp = figure(tools='xwheel_zoom,xpan',
title="",
x_axis_label='Time[sec]',
y_axis_label='Acceleration')
pp.line(t, Data[0],legend="X", line_width=1, line_color = "blue")
pp.line(t, Data[1],legend="Y", line_width=1, line_color = "red")
pp.line(t, Data[2],legend="Z", line_width=1, line_color = "green")
output_file("robot.html")
show(pp)

df = pd.Series(Data[0])
df

df.to_csv( 'data.csv' )

Loading BokehJS ...